<a href="https://colab.research.google.com/github/bimasaktikr/jobstreet-scraping/blob/main/Jobstreet_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Requirement**

In [ ]:
!pip install requests-html
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |████████████████████████████████| 140 kB 71.3 MB/s 
     |████████████████████████████████| 112 kB 71.2 MB/s 
     |████████████████████████████████| 127 kB 72.3 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=96e0e943867c179eb403b10cd2d1a7403797b69fd0d30fc4ae601c49e787f2e6
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=0799a607f34006bfea78c6c7191ad94bd1f847cf7ecc4bf1d60b8d8300cbb405
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

**Import Library yang dibutuhkan**



In [ ]:
from requests_html import HTMLSession
import csv
from datetime import datetime
import pandas as pd


**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Ambil Link Setiap halaman**

In [ ]:
s = HTMLSession()

def get_job_links(page):
    url = f'https://www.jobstreet.co.id/id/job-search/job-vacancy.php?pg={page}'
    links = []
    r = s.get(url)
    jobs = r.html.find('div.sx2jih0 h1.sx2jih0')
    for item in jobs:
        links.append(item.find('a', first=True).attrs['href'])
    return(links)

**Parse Setiap Lowongan**

In [ ]:
# parse_jobs()
def parse_jobs(url):
    s = HTMLSession()
    # r = s.get('https://www.jobstreet.co.id/'+ url)
    jobs = {}

    while True:
        r = s.get('https://www.jobstreet.co.id/'+ url)
        # r = s.get('https://www.jobstreet.co.id/id/job/management-trainee-4035514?jobId=jobstreet-id-job-4035514&sectionRank=38&token=0~1b906f5c-10dc-4c4b-b1a5-d9f1e585ad2e&searchPath=%2Fid%2Fjob-search%2Fjob-vacancy.php%3Fpg%3D2&fr=SRP%20View%20In%20New%20Tab')

        # company = r.html.find('div.sx2jih0.zcydq86m span.sx2jih0.zcydq84u._18qlyvc0._18qlyvc1x._18qlyvc2._1d0g9qk4._18qlyvcb')
        company = r.html.find('div.sx2jih0._17fduda0._17fduda6 div.sx2jih0.zcydq86m span.sx2jih0.zcydq84u._18qlyvc0._18qlyvc1x._18qlyvc2._1d0g9qk4._18qlyvcb')
        if company :                                                                                
            jobs['company'] = company[0].text
            break
        # else :
        #     jobs['company'] = 'NA'
        #     break
            

    position = r.html.find('div.sx2jih0.zcydq86m h1.sx2jih0._18qlyvc0._18qlyvch._1d0g9qk4._18qlyvcp._18qlyvc1x')
    if position:
        jobs['position'] = position[0].text

    multiplelocation = r.html.find('div.sx2jih0.zcydq8ci span.sx2jih0.zcydq84u._18qlyvc0._18qlyvc1x._18qlyvc1._18qlyvca')
    if multiplelocation:
        # print("multiple:"+ multiplelocation[0].text)
        location = r.html.find('div.sx2jih0.zcydq856.zcydq8fa.zcydq85a span.sx2jih0.zcydq84u._18qlyvc0._18qlyvc1x._18qlyvc1._18qlyvca')
        # location = r.html.find('div.sx2jih0.zcydq86i.zcydq87i span.sx2jih0.zcydq84u._18qlyvc0._18qlyvc1x._18qlyvc1._18qlyvca')
        # print(len(location))
        for i in range(len(location)):
            # print("lokasi:" + location[i].text)
            limit = ['Beberapa lokasi kerja']
            multiLoc = []
            if str(location[i].text) == limit[0]:
                # print ("TRUE")
                indexBeberapa = i
                # print(indexBeberapa)
                for j in range(0, indexBeberapa):
                    # print(location[j].text)
                    multiLoc.append(location[j].text)
                break
        locationJoin = ','.join(str(e) for e in multiLoc)
        # print(locationJoin)
        # locationJoin =''
        # for x in multiLoc:
        #     locationJoin += ', '+x

        jobs['location'] = locationJoin
    else:
        location =r.html.find('div.sx2jih0.zcydq856 span.sx2jih0.zcydq84u._18qlyvc0._18qlyvc1x._18qlyvc1._18qlyvca')
        if location:
            jobs['location'] = location[3].text
        else:
            jobs['location'] = "NA"   
    # print (jobs)
    
    span = r.html.find('div.sx2jih0.zcydq856.zcydq8fe._17fduda27')
    # spanresults = {}
    for i in span:
        preText = i.text
        afterText = preText.split('\n')
        keyword = ['Tingkat Pekerjaan', 
                    'Kualifikasi', 
                    'Pengalaman Kerja', 
                    'Jenis Pekerjaan', 
                    'Spesialisasi Pekerjaan',
                    'Industri'] 
        for j in range(len(afterText)):
            # print(afterText[j]+"-----")
            if afterText[j] == keyword[0]:
                position_key = afterText[j]
                position_value = afterText[j+1]
                jobs[position_key] = position_value
            elif afterText[j] == keyword[1]:
                kualifikasi_key = afterText[j]
                kualifikasi_value = afterText[j+1]
                jobs[kualifikasi_key] = kualifikasi_value
            elif afterText[j] == keyword[2]:
                pengalaman_key = afterText[j]
                pengalaman_value = afterText[j+1]
                jobs[pengalaman_key] = pengalaman_value
            elif afterText[j] == keyword[3]:
                jenis_key = afterText[j]
                jenis_value = afterText[j+1]
                jobs[jenis_key] = jenis_value
            elif afterText[j] == keyword[4]:
                spesialisasi_key = afterText[j]
                spesialisasi_value = afterText[j+1]
                jobs[spesialisasi_key] = spesialisasi_value
            elif afterText[j] == keyword[5]:
                industry_key = afterText[j]
                industry_value = afterText[j+1]
                jobs[industry_key] = industry_value

        #Handle Jika ada 1 key yang hilang
        if not keyword[2] in jobs:
            jobs[keyword[2]] = "Tidak terspesifikasi"
        if not keyword[3] in jobs:
            jobs[keyword[3]] = "Tidak terspesifikasi"
        if not keyword[4] in jobs:
            jobs[keyword[4]] = "Tidak terspesifikasi"
        if not keyword[5] in jobs:
            jobs[keyword[5]] = "Tidak terspesifikasi"
    print (jobs)         
    # currentDate = datetime.now()
    return jobs

**Mengambil Range Page Jobstreet**

In [ ]:
def getPageRange():
    s = HTMLSession()
    url = f'https://www.jobstreet.co.id/id/job-search/job-vacancy.php'
    r = s.get(url)
    pages = []
    page = r.html.find('div.sx2jih0.zcydq86e select.sx2jih0.sx2jih1.sx2jih6.sx2jih7.sx2jiha.zcydq8p.zcydq89i.zcydq85u.zcydq85.tfxl6i0._3dxvbx0._3dxvbx1._18qlyvc0._18qlyvc1x._18qlyvc1._18qlyvca._18qlyvc22._18qlyvc14.zcydq832')
    if page:
        strpage = str(page[0].text)
        afterText = strpage.split('\n')
        awal = afterText[0]
        akhir = afterText[len(afterText)-1]
    return (awal, akhir)

**Membagi Scraper Setiap 100 Halaman**

In [ ]:
def chunkPage():
    chunkSize = 5
    pageRange = getPageRange()
    tempList = []
    tempList.append(1)    
    for i in range(int(pageRange[0]), int(pageRange[1])):
       if (i % 100 == 0):
            tempList.append(i+1)
    if not ((int(pageRange[1]))%100==0):
        tempList.append(int(pageRange[1]))

    return tempList

**Web Scrap Setiap 100 Halaman**

In [ ]:
def chunkedScrapper():
    PageList = chunkPage()
    print(PageList)
    for i in range(0, len(PageList)+1):
        scrap(PageList[i], PageList[i+1])


**Eksport Hasil Scraping**

In [ ]:
def save_csv(results,start, end):
    keys = results[0].keys()
    # currentDate = datetime.today().strftime('%Y-%m-%d-%H_%M_%S')
    currentDate = datetime.today().strftime('%Y-%m-%d-%H')
    strDate = str(currentDate)
    
    ##save to Drive##
    path = '/content/drive/MyDrive/0000. Jobs Project/Output/jobs-jobstreet-ID-'+ str(start) + "_" + str(end) + '.csv'
    with open(path, 'w', encoding = 'utf-8-sig') as f:
      df = pd.DataFrame(results)
      df.to_csv(f)
    
    ##save to CSV Local##
    # with open('jobs-jobstreet-ind' + strDate + '.csv', 'w', encoding="utf-8") as f:
    #     dict_writer = csv.DictWriter(f, keys)
    #     dict_writer.writeheader()
    #     dict_writer.writerows(results)

In [ ]:
def scrap(start, end):
    results = []
    for i in range (start,end):
        urls = get_job_links(i)
        for url in urls:
            results.append(parse_jobs(url)) 
        print('Total Results: ', len(results))
        # time.sleep(3)
    save_csv(results, start, end)

In [ ]:
chunkedScrapper()

[1, 101, 201, 301, 401, 501, 601, 701, 801, 901, 1001, 1101, 1201, 1301, 1327]
{'company': 'PT Dongil Rubber Belt Indonesia', 'position': 'Import & Sales Admin Staff', 'location': 'Jakarta Raya', 'Tingkat Pekerjaan': 'Pegawai (non-manajemen & non-supervisor)', 'Kualifikasi': 'Sertifikat Professional, D3 (Diploma), D4 (Diploma)', 'Pengalaman Kerja': '2 tahun', 'Jenis Pekerjaan': 'Penuh Waktu', 'Spesialisasi Pekerjaan': 'Sumber Daya Manusia/Personalia, Staf / Administrasi umum', 'Industri': 'Automobil/Mesin Tambahan Automotif/Kendaraan'}
{'company': 'PT Biotech Indo Gemilang', 'position': 'ADMINISTRASI PEMBELIAN', 'location': 'Jakarta Barat', 'Tingkat Pekerjaan': 'Pegawai (non-manajemen & non-supervisor)', 'Kualifikasi': 'Sertifikat Professional, D3 (Diploma), D4 (Diploma), Sarjana (S1)', 'Pengalaman Kerja': '1 tahun', 'Jenis Pekerjaan': 'Penuh Waktu', 'Spesialisasi Pekerjaan': 'Manufaktur, Pembelian/Manajemen Material', 'Industri': 'Umum & Grosir'}
{'company': 'PT Surya Cipta Sempurna',